In [ ]:
#Imports
import os
import copy
import pandas as pd
import tensorflow as tf
from keras import backend as K
import scipy.stats as stats

import sys
sys.path.insert(0, '..')
from utils.modelgenerator import *
from utils.modelhandler import *
from utils.datahandler import *
from utils.attackhandler import *

import logging
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
logging.getLogger('tensorflow').setLevel(logging.ERROR)

In [ ]:
file_path = os.path.join(os.path.normpath(os.path.dirname(os.path.dirname(os.getcwd()))), 'data/3final_data/Final_Energy_dataset.csv')
cwd = os.path.normpath(os.getcwd())
use_sin_cos_features = True

# User indices based on clusters
user_indices = [16, 24] # Cluster 2: [16,24] / Cluster 4: [1, 11, 12, 27] / Cluster 10: [2, 4, 6, 9, 10, 14, 15, 18, 25, 30]

# Data processing parameters
sequence_length = 49
batch_size = 16

# Initialize necessary objects
dh = Datahandler()
m1 = ModelGenerator()
mh = Modelhandler()

# Load and prepare data
print("Loading and preparing data...")
df_array = load_and_prepare_data(file_path, user_indices, columns_filter_prefix="load")

# Apply sine and cosine transformations if the flag is set to True
if use_sin_cos_features:
    for i in range(len(df_array)):
        df_array[i] = create_sin_cos_features(df_array[i])

# Split the data into training, validation, and test sets
print("Splitting data into train, validation, and test sets...")
X_train, y_train, X_val, y_val, X_test, y_test = split_data(df_array, sequence_length, batch_size, dh)


X_train_raw = X_train

loss = tf.keras.losses.MeanSquaredError()
metrics=[tf.keras.metrics.RootMeanSquaredError(), tf.keras.metrics.MeanAbsoluteError()]
callbacks = get_callbacks()

In [ ]:
#No Attack

#NORMAL DISTRIBUTION - Scale 1
attack = "FL_Load_noBACKDOOR_noBACKDOORNoise"
X_train = copy.deepcopy(X_train_raw)
#X_train = backdoor_attack_at_time(X_train, noise_scale=1.0)

plot_impact_of_attack_noise(X_train_raw, X_train, user="user1", features=True)

#Run Tests
run_federated_training(df_array, X_train, y_train, X_val, y_val, X_test, y_test, callbacks, m1, mh, attack, cwd, loss, metrics)
aggregated_results, all_results = run_federated_local_evaluation(df_array, X_train, y_train, X_val, y_val, X_test, y_test, callbacks, m1, mh, attack, cwd, loss, metrics)

save_dictionaries([
    (f"{attack}_aggregated_results", aggregated_results), 
    (f"{attack}_all_results", all_results)], folder_name="results/")